In [3]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests

# Import the API key.
from config import weather_api_key

# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime

In [4]:
# Starting the URL for Weather Map API Call.
url = "https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid="+ weather_api_key

In [5]:
# Create a set of random latitudes and longitude combinations.
lats = np.random.uniform(low = -90.000, high = 90.000, size = 1500)
lngs = np.random.uniform(low = -180.000, high = 180.000, size = 1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [6]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [7]:
# Use the print() function to display the latitude and longitude combinations.
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

-56.331844279262874 -79.81915685494481
-84.37032995632573 112.12238695836908
-57.610541648353404 -97.4211571109834
28.83561040324615 -78.17633686165345
-41.42953574298542 -123.19477392656532
-66.06271042191153 -84.74239891862909
-28.082812120420293 63.20721849622248
0.2829811332906047 -37.04348584788491
50.63630411399777 -58.80090115139487
66.5523447561026 -150.22833508735926
-83.69094092705268 -85.30507088147614
-71.795081751332 35.79914926380124
-55.98561158284461 38.54045216194865
-19.633810796313114 130.6845838494662
81.53723048615385 -163.9426618565897
-2.8117625588519104 -44.044246152652164
24.469565051774026 149.4376816524591
-65.278603035708 -21.78336749510919
22.0707243089673 44.57823411135047
-4.429551797017552 -103.77097849850794
-9.71296015853639 61.536254696096705
25.39680286930073 -152.64028715947924
54.42803989931909 106.31252197888227
-28.416641263902328 -164.7846615136216
-78.57519015261173 160.61866180675764
-1.8614046387416892 -177.5613004881114
-69.5221710660725 -85

In [8]:
# Create a list for holding the cities.
cities = []

# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, the we will add it to the cities list.
    if city not in cities:
        cities.append(city)
        
# Print the city count to confirm sufficient count.
len(cities)

621

In [9]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [10]:
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | punta arenas
Processing Record 2 of Set 1 | albany
Processing Record 3 of Set 1 | high rock
Processing Record 4 of Set 1 | rikitea
Processing Record 5 of Set 1 | mahebourg
Processing Record 6 of Set 1 | trairi
Processing Record 7 of Set 1 | saint-augustin
Processing Record 8 of Set 1 | college
Processing Record 9 of Set 1 | ushuaia
Processing Record 10 of Set 1 | port alfred
Processing Record 11 of Set 1 | kununurra
Processing Record 12 of Set 1 | barrow
Processing Record 13 of Set 1 | icatu
Processing Record 14 of Set 1 | katsuura
Processing Record 15 of Set 1 | chuy
Processing Record 16 of Set 1 | riyadh
Processing Record 17 of Set 1 | puerto ayora
Processing Record 18 of Set 1 | victoria
Processing Record 19 of Set 1 | kahului
Processing Record 20 of Set 1 | kachug
Processing Record 21 of Set 1 | avarua
Processing Record 22 of Set 1 | bluff
Processing Record 23 of Set 1 | vaitupu
City not found. Skipping...
Processing Record 24 of Set 1 | kingisepp
Pro

Processing Record 41 of Set 4 | zavodskoy
Processing Record 42 of Set 4 | mincivan
Processing Record 43 of Set 4 | chateaubelair
Processing Record 44 of Set 4 | arraial do cabo
Processing Record 45 of Set 4 | port macquarie
Processing Record 46 of Set 4 | villarrobledo
Processing Record 47 of Set 4 | coffs harbour
Processing Record 48 of Set 4 | havre-saint-pierre
Processing Record 49 of Set 4 | fort nelson
Processing Record 50 of Set 4 | littleton
Processing Record 1 of Set 5 | makung
City not found. Skipping...
Processing Record 2 of Set 5 | wajir
Processing Record 3 of Set 5 | abu samrah
Processing Record 4 of Set 5 | lerwick
Processing Record 5 of Set 5 | vestmannaeyjar
Processing Record 6 of Set 5 | kattivakkam
Processing Record 7 of Set 5 | sorland
Processing Record 8 of Set 5 | attawapiskat
City not found. Skipping...
Processing Record 9 of Set 5 | opuwo
Processing Record 10 of Set 5 | killybegs
Processing Record 11 of Set 5 | pevek
Processing Record 12 of Set 5 | mandalgovi
Pro

Processing Record 32 of Set 8 | ngukurr
City not found. Skipping...
Processing Record 33 of Set 8 | marsa matruh
Processing Record 34 of Set 8 | toktogul
Processing Record 35 of Set 8 | spisska nova ves
Processing Record 36 of Set 8 | puerto madryn
Processing Record 37 of Set 8 | vestbygda
City not found. Skipping...
Processing Record 38 of Set 8 | haines junction
Processing Record 39 of Set 8 | northam
Processing Record 40 of Set 8 | tingrela
City not found. Skipping...
Processing Record 41 of Set 8 | suhut
Processing Record 42 of Set 8 | flinders
Processing Record 43 of Set 8 | buchanan
Processing Record 44 of Set 8 | yashkul
Processing Record 45 of Set 8 | sao miguel do oeste
City not found. Skipping...
Processing Record 46 of Set 8 | san luis
Processing Record 47 of Set 8 | tupik
Processing Record 48 of Set 8 | ribeira grande
Processing Record 49 of Set 8 | vasai
Processing Record 50 of Set 8 | aquiraz
Processing Record 1 of Set 9 | cocal
Processing Record 2 of Set 9 | rehoboth
Pro

Processing Record 14 of Set 12 | latung
Processing Record 15 of Set 12 | malayal
Processing Record 16 of Set 12 | el tigre
Processing Record 17 of Set 12 | lodwar
Processing Record 18 of Set 12 | kawalu
Processing Record 19 of Set 12 | santiago
Processing Record 20 of Set 12 | bolungarvik
City not found. Skipping...
Processing Record 21 of Set 12 | urdzhar
City not found. Skipping...
Processing Record 22 of Set 12 | batagay-alyta
Processing Record 23 of Set 12 | rio verde de mato grosso
Processing Record 24 of Set 12 | muli
Processing Record 25 of Set 12 | khatanga
Processing Record 26 of Set 12 | myanaung
Processing Record 27 of Set 12 | zhanaozen
Processing Record 28 of Set 12 | los llanos de aridane
Processing Record 29 of Set 12 | kutum
Processing Record 30 of Set 12 | gangapur
Processing Record 31 of Set 12 | oranjemund
Processing Record 32 of Set 12 | yertsevo
Processing Record 33 of Set 12 | sovetskiy
Processing Record 34 of Set 12 | indiana
Processing Record 35 of Set 12 | kiun

In [11]:
len(city_data)

569

In [12]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Punta Arenas,-53.1500,-70.9167,283.21,40,75,11.32,CL,2022-11-19 22:38:19
1,Albany,42.6001,-73.9662,274.38,51,65,2.12,US,2022-11-19 22:44:09
2,High Rock,26.6208,-78.2833,298.16,73,77,7.75,BS,2022-11-19 22:44:10
3,Rikitea,-23.1203,-134.9692,294.97,66,96,3.98,PF,2022-11-19 22:40:57
4,Mahebourg,-20.4081,57.7000,294.31,83,20,1.54,MU,2022-11-19 22:41:09
5,Trairi,-3.2778,-39.2689,299.22,80,34,5.38,BR,2022-11-19 22:41:27
6,Saint-Augustin,51.2260,-58.6502,263.20,96,60,2.67,CA,2022-11-19 22:44:12
7,College,64.8569,-147.8028,262.92,86,75,0.00,US,2022-11-19 22:44:12
8,Ushuaia,-54.8000,-68.3000,281.96,57,40,9.26,AR,2022-11-19 22:40:56
9,Port Alfred,-33.5906,26.8910,286.03,94,5,0.45,ZA,2022-11-19 22:44:13


In [17]:
new_column_order = ['City', 'Country', 'Date', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed']
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Punta Arenas,CL,2022-11-19 22:38:19,-53.1500,-70.9167,283.21,40,75,11.32
1,Albany,US,2022-11-19 22:44:09,42.6001,-73.9662,274.38,51,65,2.12
2,High Rock,BS,2022-11-19 22:44:10,26.6208,-78.2833,298.16,73,77,7.75
3,Rikitea,PF,2022-11-19 22:40:57,-23.1203,-134.9692,294.97,66,96,3.98
4,Mahebourg,MU,2022-11-19 22:41:09,-20.4081,57.7000,294.31,83,20,1.54
5,Trairi,BR,2022-11-19 22:41:27,-3.2778,-39.2689,299.22,80,34,5.38
6,Saint-Augustin,CA,2022-11-19 22:44:12,51.2260,-58.6502,263.20,96,60,2.67
7,College,US,2022-11-19 22:44:12,64.8569,-147.8028,262.92,86,75,0.00
8,Ushuaia,AR,2022-11-19 22:40:56,-54.8000,-68.3000,281.96,57,40,9.26
9,Port Alfred,ZA,2022-11-19 22:44:13,-33.5906,26.8910,286.03,94,5,0.45


In [19]:
# Create the output file (CSV).
output_data_file = "cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")